In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_curve, auc, precision_score, recall_score, f1_score
import statsmodels.api as sm
import matplotlib.pyplot as plt 
import itertools
import math
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_excel('Bank Marketing Data_raw.xlsx')

## All data (categorical/numerical)

In [4]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


### One-hot encoding the data

In [5]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the encoder with drop_first=True
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit and transform the categorical columns
one_hot_encoded = encoder.fit_transform(df[['job', 'marital', 'education', 'contact', 'month', 'poutcome']])

# Create a DataFrame with the encoded columns
one_hot_df = pd.DataFrame(one_hot_encoded, 
                          columns=encoder.get_feature_names_out(['job', 'marital', 'education', 'contact', 'month', 'poutcome']))


Concatenate encoded DataFrame with the rest of the data

In [6]:
# Drop the original categorical columns from `df`
df_other = df.drop(['job', 'marital', 'education', 'contact', 'month', 'poutcome'], axis=1)

# Concatenate the one-hot encoded columns with the rest of the data
final_df = pd.concat([df_other, one_hot_df], axis=1)

# Reorder columns so that 'y' is the last column
final_df = final_df[[col for col in final_df.columns if col != 'y'] + ['y']]

# Convert 'y', 'default', 'housing', and 'loan' to numerical
final_df[['y', 'default', 'housing', 'loan']] = final_df[['y', 'default', 'housing', 'loan']].replace({'yes': 1, 'no': 0})

# Resulting DataFrame with one-hot encoding and other transformations
df_encoded = final_df
df_encoded

C:\Users\jedre\AppData\Local\Temp\ipykernel_27664\786018203.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df[['y', 'default', 'housing', 'loan']] = final_df[['y', 'default', 'housing', 'loan']].replace({'yes': 1, 'no': 0})


,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown,y
0,30,0,1787,0,0,19,79,1,-1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
1,33,0,4789,1,1,11,220,1,339,4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,35,0,1350,1,0,16,185,1,330,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,30,0,1476,1,1,3,199,4,-1,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,59,0,0,1,0,5,226,1,-1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,0,-333,1,0,30,329,5,-1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4517,57,1,-3313,1,1,9,153,1,-1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4518,57,0,295,0,0,19,151,11,-1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4519,28,0,1137,0,0,6,129,4,211,3,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [7]:
df_encoded.to_excel('NewEncodedData.xlsx')

In [10]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  4521 non-null   int64  
 1   default              4521 non-null   int64  
 2   balance              4521 non-null   int64  
 3   housing              4521 non-null   int64  
 4   loan                 4521 non-null   int64  
 5   day                  4521 non-null   int64  
 6   duration             4521 non-null   int64  
 7   campaign             4521 non-null   int64  
 8   pdays                4521 non-null   int64  
 9   previous             4521 non-null   int64  
 10  job_blue-collar      4521 non-null   float64
 11  job_entrepreneur     4521 non-null   float64
 12  job_housemaid        4521 non-null   float64
 13  job_management       4521 non-null   float64
 14  job_retired          4521 non-null   float64
 15  job_self-employed    4521 non-null   f